### Figure 1: Predicted images

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import metpy.plots.ctables
from utils.data import load_tc_data
from utils.plot import make_cmap

# TODO: because i'm using imshow the lats need to be flipped - normally would be fine but because lats are flipped and imshow doesn't know this
# the TCs are plotted spinning the wrong way

sns.set_style("white")
sns.set_palette(sns.color_palette("Paired"))
sns.set_palette(sns.color_palette("Set2"))

In [2]:
# define cmap
precip_cmap,precip_norm = make_cmap()

In [3]:
def regrid(array):
        hr_array = np.zeros((100,100))
        for i in range(10):
                for j in range(10):
                        i1 = i*10
                        i2 = (i+1)*10
                        j1 = j*10
                        j2 = (j+1)*10
                        hr_array[i1:i2,j1:j2] = array[i,j]
        return hr_array

In [4]:
def plot_predictions(inputs,inputs_x,pred_gan,pred_gan_x,pred_gan_combined,pred_gan_x_combined,real,real_x,storms,range_,plot='save'):
	if plot == 'save':
			fig, axes = plt.subplots(n, m, figsize=(4*m, 4*n), sharey=True)
	else:
			print('show')
			fig, axes = plt.subplots(n, m, figsize=(2*m, 2*n), sharey=True)

	pal = sns.cubehelix_palette(10, rot=-.25, light=.7,as_cmap=True)

	for i in range(m):
		j = 0
		
		storm = storms[i]
		if storm in [3014,3487,3757,2651,3487,680,1438,3738]:
			print('THE STORM IS:',storm)
			print('storm max: ',real_x[storm].max())
			print('gan max: ',pred_gan_x[storm].max())
			print('gan combined max: ',pred_gan_x_combined[storm].max())
			print('inputs max: ', np.nanmax(inputs_x[storm]))
			print('\n')
		# norm = colors.Normalize(*range_)
		norm = precip_norm
		if i in [0,1,2]:
			axes[j,i].imshow(regrid(inputs[storm]), interpolation='nearest', norm=norm,extent=None,cmap=precip_cmap)
			axes[j,i].invert_yaxis()
			axes[j+1,i].imshow(pred_gan[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+1,i].invert_yaxis()
			axes[j+2,i].imshow(pred_gan_combined[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+2,i].invert_yaxis()
			im = axes[j+3,i].imshow(real[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+3,i].invert_yaxis()
		else:
			axes[j,i].imshow(regrid(inputs_x[storm]), interpolation='nearest', norm=norm,extent=None,cmap=precip_cmap)
			axes[j,i].invert_yaxis()
			axes[j+1,i].imshow(pred_gan_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+1,i].invert_yaxis()
			axes[j+2,i].imshow(pred_gan_x_combined[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+2,i].invert_yaxis()
			im = axes[j+3,i].imshow(real_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+3,i].invert_yaxis()

		axes[j,i].set(xticklabels=[])
		axes[j,i].set(yticklabels=[])
		axes[j+1,i].set(xticklabels=[])
		axes[j+1,i].set(yticklabels=[])
		axes[j+2,i].set(xticklabels=[])
		axes[j+2,i].set(yticklabels=[])
		axes[j+3,i].set(xticklabels=[])
		axes[j+3,i].set(yticklabels=[])


		axes[j,0].set_ylabel('LR input',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+1,0].set_ylabel('WGAN',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+2,0].set_ylabel('WGAN multi',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+3,0].set_ylabel('HR obs',horizontalalignment='left',rotation=0,fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')

		axes[3,j].set_xlabel('-',fontsize=24,labelpad=5)
		axes[3,j+1].set_xlabel('-',fontsize=24,labelpad=5)
		axes[3,j+2].set_xlabel('-',fontsize=24,labelpad=5)
		axes[3,j+3].set_xlabel('Vayu (2019; NI)',fontsize=24,labelpad=5)


	fig.subplots_adjust(right=0.8)
	# cbar_ax = fig.add_axes([1, 0.15, 0.05, 0.7])
	cbar_ax = fig.add_axes([1, 0.031, 0.03, 0.936])
	levels=[0, 1, 2, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 70, 100, 150]
	cb = fig.colorbar(im, cax=cbar_ax,shrink=0.01,ticks=levels)
	cb.ax.tick_params(labelsize=36)
	fig.tight_layout()

	if plot == 'save':
			plt.savefig('figure_1.png',bbox_inches='tight',dpi=600)
			plt.clf()
	else:
			plt.show()
	

In [5]:
# load data
real,inputs,pred_gan,pred_gan_combined,pred_gan_ensemble,pred_gan_ensemble_combined,meta,meta_combined = load_tc_data(set='validation',results='combined')
# real_x,inputs_x,pred_cnn_x,pred_vaegan_x,pred_gan_x,pred_vaegan_ensemble_x,pred_gan_ensemble_x,meta_extreme_valid = load_tc_data(set='extreme_valid',results='combined')
# real_x,inputs_x,pred_cnn_x,pred_vaegan_x,pred_gan_x,pred_vaegan_ensemble_x,pred_gan_ensemble_x,meta_extreme_valid = load_tc_data(set='extreme_test',results='combined')


# meta = pd.read_csv('/user/work/al18709/tc_data_mswep/valid_meta.csv')
# meta_extreme = pd.read_csv('/user/work/al18709/tc_data_mswep/extreme_valid_meta.csv')
# meta_extreme_test = pd.read_csv('/user/work/al18709/tc_data_mswep/extreme_test_meta.csv')
# meta_test = pd.read_csv('/user/work/al18709/tc_data_mswep/test_meta.csv')
# meta_train = pd.read_csv('/user/work/al18709/tc_data_mswep/train_meta.csv')
# meta_valid = pd.read_csv('/user/work/al18709/tc_data_mswep/valid_meta.csv')

In [6]:
# take only top 100 extremes
# inputs_x_max = np.max(inputs_x,axis=(1,2))
# max_idx = list(np.argpartition(inputs_x_max, -100)[-100:])

# redefine extremes by only the most extreme samples
# inputs_hr_x = inputs_hr_x[max_idx]
# print(np.mean(np.abs(np.max(pred_cnn_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))
# print(np.mean(np.abs(np.max(pred_vaegan_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))
# print(np.mean(np.abs(np.max(pred_gan_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))

# print(np.max(np.abs(np.max(pred_cnn_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))
# print(np.max(np.abs(np.max(pred_vaegan_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))
# print(np.max(np.abs(np.max(pred_gan_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))

# print(list(np.max(real_x[max_idx],axis=(1,2))))
# print(list(np.max(pred_cnn_x[max_idx],axis=(1,2))))

# idx = list(np.argpartition(np.max(real_x[max_idx],axis=(1,2)), -10)[-10:])
# print(np.max(real_x[max_idx][idx],axis=(1,2)))
# print(np.max(pred_cnn_x[max_idx][idx],axis=(1,2)))
# print(np.max(pred_vaegan_x[max_idx][idx],axis=(1,2)))
# print(np.max(pred_gan_x[max_idx][idx],axis=(1,2)))
# print('input max',np.max(inputs_x[max_idx][idx],axis=(1,2)))
# print('real max',np.max(real_x[max_idx][idx],axis=(1,2)))
# print(idx)

# print(np.mean(np.abs(np.max(pred_cnn_x[max_idx][idx],axis=(1,2))-np.max(real_x[max_idx][idx],axis=(1,2)))))
# print(np.mean(np.abs(np.max(pred_vaegan_x[max_idx][idx],axis=(1,2))-np.max(real_x[max_idx][idx],axis=(1,2)))))
# print(np.mean(np.abs(np.max(pred_gan_x[max_idx][idx],axis=(1,2))-np.max(real_x[max_idx][idx],axis=(1,2)))))

# idx = list(np.argpartition(np.max(real_x[max_idx],axis=(1,2)), -10)[:90])
# print(idx)
# print(np.mean(np.abs(np.max(pred_cnn_x[max_idx][idx],axis=(1,2))-np.max(real_x[max_idx][idx],axis=(1,2)))))
# print(np.mean(np.abs(np.max(pred_vaegan_x[max_idx][idx],axis=(1,2))-np.max(real_x[max_idx][idx],axis=(1,2)))))
# print(np.mean(np.abs(np.max(pred_gan_x[max_idx][idx],axis=(1,2))-np.max(real_x[max_idx][idx],axis=(1,2)))))
# print(np.max(np.abs(np.max(pred_vaegan_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))
# print(np.max(np.abs(np.max(pred_gan_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))

# print(np.max(inputs_x[max_idx],axis=(1,2)))
# print(np.max(pred_cnn_x[max_idx],axis=(1,2)))
# print(np.max(pred_vaegan_x[max_idx],axis=(1,2)))
# print(np.max(pred_gan_x[max_idx],axis=(1,2)))
# pred_vaegan_x = pred_vaegan_x[max_idx]
# pred_gan_x = pred_gan_x[max_idx]
# real_x = real_x[max_idx]
# pred_vaegan_ensemble_x = pred_vaegan_ensemble_x[max_idx]
# pred_gan_ensemble_x = pred_gan_ensemble_x[max_idx]

# print(np.sort(max_idx))

In [7]:

plot = 'save'
n = 4
m = 6

# make colours
range_ = (0, 20)
storms = [4077, 4078, 4079, 4080, 4081, 4082]

# maria 8453 (8452 for unet) 8463
# haiyan 9820, 9828,  9845, 9840 9815, 9816, 9822 9809
# barb 1347 1375 1399
# georges 4082
print(inputs.shape)
print(pred_gan.shape)
print(pred_gan_combined.shape)
print(pred_gan_combined)
storms = [8453,9845,1375,4128,5358,747]
storms = [8453,9845,1375,3014,3487,3757] 
storms = [8453,9845,1375,3014,3487,3738]
# storms_extremes = [4082,5349,763]
# pred_gan_combined = 10**pred_gan_combined - 1
# inputs = 10**inputs -1
# real = 10**real -1
plot_predictions(inputs,inputs,pred_gan,pred_gan,pred_gan_combined,pred_gan_combined,real,real,storms,range_)


(15930, 10, 10)
(16253, 100, 100)
(15930, 100, 100)
[[[1.00859933e-01 4.43689810e-02 2.84850727e-02 ... 8.13658606e-02
   1.07853199e-01 1.62356302e-01]
  [2.42338957e-02 8.11791152e-03 7.56800599e-03 ... 7.67702959e-02
   8.39993087e-02 1.31148348e-01]
  [1.39578339e-02 4.04606814e-03 4.53130376e-03 ... 9.08426393e-02
   9.77359893e-02 1.04727898e-01]
  ...
  [2.76089820e-01 3.57494562e-01 4.07873464e-01 ... 2.01191843e-01
   1.91842801e-01 2.95003016e-01]
  [2.59917310e-01 2.43207029e-01 4.15812994e-01 ... 2.39468071e-01
   2.22824810e-01 3.36768060e-01]
  [2.51071509e-01 2.65873054e-01 3.55138031e-01 ... 4.37799961e-01
   4.25031937e-01 5.06800216e-01]]

 [[9.07500460e-02 5.87033309e-02 6.03610553e-02 ... 5.97163683e-01
   5.77403584e-01 5.08767655e-01]
  [3.18815983e-02 1.45860376e-02 1.49111860e-02 ... 6.27039271e-01
   4.43829026e-01 4.20561202e-01]
  [2.57978992e-02 1.22962973e-02 1.19799826e-02 ... 5.68444932e-01
   5.33957116e-01 4.16250898e-01]
  ...
  [1.75978110e+00 1.90682

/user/work/al18709/.conda/envs/aquatic/lib/python3.7/site-packages/ipykernel_launcher.py:69: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


<Figure size 1728x1152 with 0 Axes>

In [ ]:
# print(meta.shape)
# print(real.shape) 

# print(np.sum(meta['sid'] == '1992044S09181'))
# print(np.sum(meta_extreme['sid'] == '1992044S09181'))
# print(np.sum(meta_test['sid'] == '1992044S09181'))
# print(np.sum(meta_extreme_test['sid'] == '1992044S09181'))
# print(np.sum(meta_train['sid'] == '1992044S09181'))


# print(meta_extreme['sid'].values)
# for i,storm in enumerate(meta_extreme['sid'].values):
# 	pre_storm = meta_extreme['sid'].values[i-1]
# 	if storm != pre_storm:
# 		print(storm)

# Dorian in the test set
# Isaac (2012) '2012234N16315' training set NA
# Amphan 2020136N10088 ****** lost NI
# Harvey 2017228N14314 train NA
# Irma 2017242N16333 train NA
# Idai 2019063S18038 train SIO 
# Winston 2016041S14170 test SP
# Matthew 2016273N13300 extreme_test NA
# Jalawat 2012263N15141 extreme_test WP
# irene 2011233N15301 valid
# isaac 2012234N16315 train


# EP NA WP
# NIO SIO SP

# Maria 2017260N12310 validation NA
# Haiyan 2013306N07162 validation WP
# Barbara 1995189N11257 validation EP
print(np.where(meta_extreme['sid'] == '2020138N10086'))
print(np.where(meta_valid['sid'] == '2020138N10086'))
print(np.where(meta_train['sid'] == '2020138N10086'))
print(np.where(meta_test['sid'] == '2020138N10086'))
print(np.where(meta_extreme_test['sid'] == '2020138N10086'))
print(np.where(meta_extreme_valid['sid'] == '2020138N10086'))



# Georges 1998259N10335 extreme valid NA
# Daman 1992044S09181 extreme valid SP
# Haleh 2019060S10078 extreme valid SIO


# accumulated:
# Irene 2011233N15301 validation NA
# Haiyan 2013306N07162 validation WP
# Pauline 1997279N12263 extreme test EP
# Yasa 2020348S14173 SP (fiji)
# Idai 2019063S18038 train SI or Haleh 2019060S10078 extreme valid SI
# Amphan 2020136N10088 NI 2020138N10086

# check
# Harvey 2017228N14314 train NA
# Isaac 2012234N16315 train NA
# Maria 2017260N12310 validation NA
# Irma 2017242N16333 train NA
# Dorian 2019236N10314 test NA

In [ ]:
tc_list = [680,1438,2651,3014,3487,3738,3757]

for i in tc_list:
	print(meta_extreme_test['sid'][i])

In [ ]:
# 2019160N11073 - VAYU 2019 NI 3014
# 2012346S14180 - Evan 2013 SP 2857
# 2005042S12190 - Nancy 2005 SP 2651

# 3014 VAYU 3014
# 3487 Ursula 1998 SP
# 3738 Typhoon Brenden 1985 WP

# 3757 Cyclone Ofa 1990 SP

# 680


In [ ]:
print(np.max(inputs))
print(np.max(real))

In [ ]:
print(np.max(inputs_x))
print(np.max(real_x))

In [ ]:
def plot_haleh(inputs,inputs_x,pred_cnn,pred_cnn_x,pred_vaegan,pred_vaegan_x,pred_gan,pred_gan_x,real,real_x,storms,range_,plot='save'):

	fig, axes = plt.subplots(1, 3, figsize=(30, 10), sharey=True)
	pal = sns.cubehelix_palette(10, rot=-.25, light=.7,as_cmap=True)

	storm=storms
	norm = precip_norm
	axes[0].imshow(regrid(inputs_x[storm]), interpolation='nearest', norm=norm,extent=None,cmap=precip_cmap)
	axes[0].invert_yaxis()
	axes[1].imshow(pred_vaegan_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
	axes[1].invert_yaxis()
	im = axes[2].imshow(real_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
	axes[2].invert_yaxis()

	axes[0].set(xticklabels=[])
	axes[0].set(yticklabels=[])
	axes[1].set(xticklabels=[])
	axes[1].set(yticklabels=[])
	axes[2].set(xticklabels=[])
	axes[2].set(yticklabels=[])


	# axes[2].set_xlabel('Haleh (2019)',fontsize=24,labelpad=5)

	fig.subplots_adjust(right=0.8)
	cbar_ax = fig.add_axes([1, 0.031, 0.03, 0.936])
	levels=[0, 1, 2, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 70, 100, 150]
	cb = fig.colorbar(im, cax=cbar_ax,shrink=0.005,ticks=levels)
	cb.ax.tick_params(labelsize=36)
	fig.tight_layout()

	if plot == 'save':
			plt.savefig('tropical_cyclone_haleh.png',bbox_inches='tight',dpi=600)
			plt.clf()
	else:
			plt.show()
	

In [ ]:
# real_x,inputs_x,pred_cnn_x,pred_vaegan_x,pred_gan_x,pred_vaegan_ensemble_x,pred_gan_ensemble_x,meta_extreme_valid = load_tc_data(set='extreme_valid',results='test')
real_x,inputs_x,pred_cnn_x,pred_vaegan_x,pred_gan_x,pred_vaegan_ensemble_x,pred_gan_ensemble_x,meta_extreme_valid = load_tc_data(set='extreme_test',results='test')


In [ ]:
storms = [8453,9845,1375,3014,3487,3757]
storms = [8453,9845,1375,4128,5358,747]
# print(max_idx)
storms = 1624
pred_vaegan_x_max = np.max(pred_vaegan_x,axis=(1,2))
max_idx = list(np.argpartition(pred_vaegan_x_max, -10)[-10:])
print(max_idx)
range_ = (0, 20)
plot_haleh(inputs,inputs_x,pred_cnn,pred_cnn_x,pred_vaegan,pred_vaegan_x,pred_gan,pred_gan_x,real,real_x,storms,range_)
